In [107]:
import json
from pathlib import Path
import re
import json
import random

In [3]:
path = Path("../../data/gqa/data/val_all_questions.json")

In [108]:
with open(path) as f:
    question = json.load(f)
keys = list(question.keys())
subset_size = int(0.1 * len(keys))
subset_keys = random.sample(keys, subset_size)
subset = {key: question[key] for key in subset_keys}

In [106]:
# print(subset_keys[:10])
# for item in subset_keys[:10]:
#     print(subset[item])
#     print("\n")

In [109]:
for key, val in subset.items():
    print(val)
    print(val['types']['semantic'])
    break

{'semantic': [{'operation': 'select', 'dependencies': [], 'argument': 'chair (-)'}, {'operation': 'exist', 'dependencies': [0], 'argument': '?'}], 'entailed': ['05526424'], 'equivalent': ['05526423', '05526424'], 'question': 'Are there any chairs in this image?', 'imageId': '2323631', 'isBalanced': False, 'groups': {'global': None, 'local': '03-chair'}, 'answer': 'no', 'semanticStr': 'select: chair (-)->exist: ? [0]', 'annotations': {'answer': {}, 'question': {}, 'fullAnswer': {}}, 'types': {'detailed': 'existC', 'semantic': 'obj', 'structural': 'verify'}, 'fullAnswer': 'No, there are no chairs.'}
obj


In [110]:
print(len(subset))

201185


# first round of filtering

In [111]:
# first filter
filtered_dic = {}
multiple_mention_dic = {}
half_or_no_mention_dic = {}
global_dic = {}
noun_not_in_q = {}

pattern = r"(\w+)\s*\((\d+(?:\s*,\s*\d+)*)\)" # should be a correct one
for key, val in subset.items():
    question = val['question']
    question_type = val['types']['semantic']
    if question_type != 'global':
        overall_flag = True
        multiple_mention_flag = False
        half_or_no_mention_flag = False
        noun_not_in_q_flag = False
        semantics = val.get('semantic', [])
        for operation in semantics:
            if operation['operation'] == 'select':
                text = operation['argument']
                match = re.search(pattern, text)
                if match: 
                    noun = match.group(1)
                    if noun not in question: 
                        overall_flag = False
                        noun_not_in_q_flag = True
                    else:
                        number = match.group(2)
                        number_list = [n.strip() for n in numbers.split(',')]
                        if len(number_list) > 1:
                            # as long as there is a multiple mention, add this case
                            multiple_mention_flag = True
                            overall_flag = False
                else: 
                    overall_flag = False
                    half_or_no_mention_flag = True
        if overall_flag:
            filtered_dic[key] = val
        elif multiple_mention_flag:
            multiple_mention_dic[key] = val
        elif half_or_no_mention_flag:
            half_or_no_mention_dic[key] = val  
        elif noun_not_in_q_flag:
            noun_not_in_q[key] = val
    else:
        global_dic[key] = val        

In [115]:
print(len(subset))
print(len(filtered_dic))
print(len(multiple_mention_dic))
print(len(half_or_no_mention_dic))
print(len(global_dic))
print(len(noun_not_in_q))

201185
141487
0
51742
3106
4850


In [113]:
final = len(subset) - (len(filtered_dic) + len(multiple_mention_dic) + len(half_or_no_mention_dic) + len(global_dic))
final

4850

In [79]:
for key, val in filtered_dic.items():
    print(key)
    print(val)
    break

05451384
{'semantic': [{'operation': 'select', 'dependencies': [], 'argument': 'cat (1324563)'}, {'operation': 'filter color', 'dependencies': [0], 'argument': 'brown'}, {'operation': 'relate', 'dependencies': [1], 'argument': 'blanket,under,s (-)'}, {'operation': 'exist', 'dependencies': [2], 'argument': '?'}], 'entailed': ['05451386', '05451385'], 'equivalent': ['05451384'], 'question': 'Are there blankets under the brown cat?', 'imageId': '2382986', 'isBalanced': False, 'groups': {'global': None, 'local': '13-cat_blanket'}, 'answer': 'no', 'semanticStr': 'select: cat (1324563)->filter color: brown [0]->relate: blanket,under,s (-) [1]->exist: ? [2]', 'annotations': {'answer': {}, 'question': {'5:7': '1324563'}, 'fullAnswer': {'4': '1324576', '7': '1324563'}}, 'types': {'detailed': 'existRelSC', 'semantic': 'rel', 'structural': 'verify'}, 'fullAnswer': 'No, there is a towel under the cat.'}


In [114]:
# store as json file
with open('filtered_dic.json', 'w') as f1:  # 'wb' means write in binary mode
    json.dump(filtered_dic, f1, indent=4)
# with open(filename, 'wb') as f2:  # 'wb' means write in binary mode
#     pickle.dump(multiple_mention_dic, f2)
with open('half_mention.json', 'w') as f3:  # 'wb' means write in binary mode
    json.dump(half_or_no_mention_dic, f3, indent=4)
with open('global_questions.json', 'w') as f4:  # 'wb' means write in binary mode
    json.dump(global_dic, f4, indent=4)
with open('noun_not_in_q.json', 'w') as f5:  # 'wb' means write in binary mode
    json.dump(noun_not_in_q, f5, indent=4)  

In [ ]:
# filter out the duplicated objects 

# Hypernym replacement

In [116]:
# read the hypernym file first
with open('../data/gqa_entities/noun-hypernyms.json') as f:
    hyper = json.load(f)
print(len(hyper))

933
